## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
my_start =  1
my_end   = 96

# my_start = 96
# my_end   = 96

96

In [3]:
# for my_int = my_start:my_end

my_int = my_start

while my_int <= my_end

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min   = jvalue(vars["f_min"])
    
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])
    
    U_min_def = jvalue(vars["U_min_def"]) # - my_correct*0.025
    
    psi_min_i = jvalue(vars["psi_min_i"])
    psi_min_d = jvalue(vars["psi_min_d"])
    psi_min_p = jvalue(vars["psi_min_p"])
    psi_min_b = jvalue(vars["psi_min_b"])
    psi_min_c = jvalue(vars["psi_min_c"])    
    
    psi_loss_i = jvalue(vars["psi_loss_i"])
    psi_loss_d = jvalue(vars["psi_loss_d"])
    psi_loss_p = jvalue(vars["psi_loss_p"])
    psi_loss_b = jvalue(vars["psi_loss_b"])
    psi_loss_c = jvalue(vars["psi_loss_c"])
    
    marge_dies = jvalue(vars["marge_dies"])
    
    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios
    L_C2 = reshape(mean(L_C,3),N_EMS,N_intra)

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Minimum frequency limitation
        # X1 - Minimum voltage relaxed limitation
        # X2 - Minimum voltage firm    limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        # B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s]*(1-psi_loss_p) + sum((P_dies[t,p,s,d] - psi_loss_d*1e3*ON_dies[t,d]) for d=ds) + P_bat_set[t,p]*(1-psi_loss_b) - L_C[t,p,s]*(1+psi_loss_c) - psi_loss_i*1e3) == 0
                
        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]
        
        # X1[t=ts ,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s] + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s] >= U_min_def
        # X1[t=m1s,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s]                      + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s]                      >= U_min_def
        # X2[t=m2s,p=ps],      psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*sum(P_PV[t,p,s] for s=ss)/N_scen + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*sum(L_C[t,p,s] for s=ss)/N_scen >= U_min_def
        
    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    # If Solved => Write Down the Results
    if string(status)!="Infeasible"
        write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
            status    = string(status),
            interval  = interval,
            P_bat_set = getvalue(P_bat_set[:,:]), 
            SOC_bat   = getvalue(SOC_bat[:,:]),
            P_PV_set  = getvalue(P_PV_set[:,:]),
            P_PV      = getvalue(P_PV[:,:,:]),
            ON_dies   = getvalue(ON_dies[:,:]),
            P_dies    = getvalue(P_dies[:,:,:,:]),
            P_bat_cha = getvalue(P_bat_cha[:,:]),
            P_bat_dis = getvalue(P_bat_dis[:,:]),
            X_bat     = getvalue(X_bat[:,:]),
            L_C       = L_C2)
    # If Infeasible -> Break!!!
    else 
        println("Loosening did not work -> Terminating Optimization!!!")
        break
    end
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/"*dec(N_EMS))
    
    my_int+=1
    
end

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]
Presolve removed 3667 rows and 4391 columns
Presolve time: 1.58s
Presolved: 192893 rows, 99001 columns, 682364 nonzeros
Variable types: 96409 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7645715e+07   1.054380e+07   0.000000e+00      7s
   41452    1.7474733e+07   9.279472e+05   0.000000e+00     10s
   73738    1.7434088e+07   5.916048e+05   0.000000e+00     15s
  101963    1.7341156e+07   4.125302e+05   0.000000e+00     20s
  127133    1.7124244e+07   2.155433e+05   0.000000e+00     25s
  151516    1.5817599e+07   1.006907e+06   0.000000e+00     30s
  174934  

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 115.735245314

Presolve removed 3666 rows and 4390 columns
Presolve time: 1.62s
Presolved: 192894 rows, 99002 columns, 682372 nonzeros
Variable types: 96410 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   24512    1.7454651e+07   1.282606e+06   0.000000e+00      5s
   54825    1.7433189e+07   7.742730e+05   0.000000e+00     10s
   82801    1.7391044e+07   5.933569e+05   0.000000e+00     15s
  110175    1.7272556e+07   3.524670e+05   0.000000e+00     20s
  134320    1.7035592e+07   1.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 609.135258214

Presolve removed 3662 rows and 4386 columns
Presolve time: 1.62s
Presolved: 192898 rows, 99006 columns, 682384 nonzeros
Variable types: 96414 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   24470    1.7341688e+07   1.272112e+06   0.000000e+00      5s
   54541    1.7316888e+07   7.877163e+05   0.000000e+00     10s
   81738    1.7284761e+07   5.319819e+05   0.000000e+00     15s
   95982    1.7242090e+07   4.140618e+05   0.000000e+00     20s
  114573    1.7134579e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 105.521680397

Presolve removed 3677 rows and 4401 columns
Presolve time: 2.81s
Presolved: 192883 rows, 98991 columns, 682344 nonzeros
Variable types: 96399 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7770581e+07   1.063398e+07   0.000000e+00      7s
   30543    1.7618141e+07   1.152502e+06   0.000000e+00     10s
   48416    1.7604723e+07   8.480793e+05   0.000000e+00     15s
   66502    1.7584216e+07   6.852193e+05   0.000000e+00     20s
   82320    1.7560019e+07   5.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 591.863339586

Presolve removed 3678 rows and 4402 columns
Presolve time: 1.72s
Presolved: 192882 rows, 98990 columns, 682348 nonzeros
Variable types: 96398 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   22292    1.7407878e+07   1.341763e+06   0.000000e+00      5s
   52937    1.7384541e+07   7.885012e+05   0.000000e+00     10s
   78623    1.7345485e+07   5.722634e+05   0.000000e+00     15s
  101550    1.7283383e+07   3.855649e+05   0.000000e+00     20s
  125636    1.7107755e+07   2.

...

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1444.704734975

Presolve removed 3705 rows and 4429 columns
Presolve time: 1.66s
Presolved: 192855 rows, 98963 columns, 682293 nonzeros
Variable types: 96371 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7501782e+07   1.044239e+07   0.000000e+00      6s
   45998    1.7328976e+07   8.663569e+05   0.000000e+00     10s
   74881    1.7284098e+07   5.750985e+05   0.000000e+00     15s
   99289    1.7202844e+07   4.637859e+05   0.000000e+00     20s
  123537    1.6992140e+07   2

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1168.851030165

Presolve removed 3707 rows and 4431 columns
Presolve time: 1.76s
Presolved: 192853 rows, 98961 columns, 682278 nonzeros
Variable types: 96369 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7736422e+07   1.050139e+07   0.000000e+00      6s
   47682    1.7559415e+07   8.479934e+05   0.000000e+00     10s
   75891    1.7523219e+07   6.154380e+05   0.000000e+00     15s
  100455    1.7438068e+07   3.958736e+05   0.000000e+00     20s
  125694    1.7228867e+07   2

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 440.151909421

Presolve removed 3699 rows and 4423 columns
Presolve time: 1.70s
Presolved: 192861 rows, 98969 columns, 682295 nonzeros
Variable types: 96377 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7588888e+07   1.056414e+07   0.000000e+00      6s
   44865    1.7421359e+07   8.751035e+05   0.000000e+00     10s
   71461    1.7391491e+07   6.026055e+05   0.000000e+00     15s
   94266    1.7333185e+07   4.567668e+05   0.000000e+00     20s
  118773    1.7167522e+07   2.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 694079 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [2e-01, 2e+04]
  RHS range        [5e-02, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 1074.360676116

Presolve removed 3688 rows and 4412 columns
Presolve time: 1.79s
Presolved: 192872 rows, 98980 columns, 682327 nonzeros
Variable types: 96388 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7730256e+07   1.047282e+07   0.000000e+00      6s
   41498    1.7568233e+07   9.043471e+05   0.000000e+00     10s
   71374    1.7532179e+07   6.156202e+05   0.000000e+00     15s
   94890    1.7470291e+07   4.288832e+05   0.000000e+00     20s
  120263    1.7283542e+07   2

Excessive output truncated after 527270 bytes.